In [1]:
import turicreate

In [2]:
sales = turicreate.SFrame('kc_house_data.csv')

Finished parsing file /home/arkojit/ML Regression/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.440995 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/arkojit/ML Regression/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.287296 secs.

In [3]:
import numpy as np

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe=data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray=data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions=np.dot(feature_matrix,weights)
    return(predictions)

In [7]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative=2*np.dot(errors, feature)
    return(derivative)

In [8]:
from math import sqrt

In [14]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions=predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        errors=predictions-output
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative=feature_derivative(errors, feature_matrix[:,i])
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_squares+=(derivative**2)
            # subtract the step size times the derivative from the current weight
            weights[i]-=(step_size*derivative)
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [15]:
train_data,test_data = sales.random_split(.8,seed=0)

In [16]:
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [17]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

In [18]:
simple_weights

array([-46999.88716555,    281.91211918])

In [22]:
simple_features = ['sqft_living']
my_output = 'price'
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

In [25]:
test_predictions=predict_output(test_simple_feature_matrix, simple_weights)

In [24]:
test_predictions[0]

356134.4432550024

In [27]:
test_res=test_output-test_predictions
test_RSS= (test_res*test_res).sum()

In [28]:
test_RSS

275400044902128.3

In [29]:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [30]:
weights_model_2=regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)

In [31]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)

In [32]:
test_predictions_2=predict_output(test_feature_matrix, weights_model_2)

In [33]:
test_predictions_2[0]

366651.4116294939

In [34]:
test_output[0]

310000.0

In [35]:
test_res=test_output-test_predictions_2
test_RSS= (test_res*test_res).sum()

In [36]:
test_RSS

270263443629803.56